### RAG 기본 구조 이해하기

1. 사전작업(Pre-processing): 데이터 소스를 Vector DB (저장소) 에 문서를 로드-분할-임베딩-저장 

- 1단계 문서로드(Document Load): 문서 내용을 불러옴
- 2단계 분할(Text Split): 문서를 특정 기준(Chunk) 으로 분할
- 3단계 임베딩(Embedding): 분할된(Chunk) 를 임베딩하여 저장
- 4단계 벡터DB 저장: 임베딩된 Chunk 를 DB에 저장

2. RAG 수행(RunTime) - 5~8 단계

- 5단계 검색기(Retriever): 쿼리(Query) 를 바탕으로 DB에서 검색하여 결과를 가져오기 위하여 리트리버를 정의
- 6단계 프롬프트: RAG 를 수행하기 위한 프롬프트를 생성. 프롬프트의 context 에는 문서에서 검색된 내용이 입력됨. 프롬프트 엔지니어링을 통하여 답변의 형식을 지정 가능
- 7단계 LLM: 모델을 정의 (GPT-3.5, GPT-4, Claude, etc..)
- 8단계 Chain: 프롬프트 - LLM - 출력 에 이르는 체인을 생성

## 환경설정


In [1]:
pip install -U langchain langchain-openai

Defaulting to user installation because normal site-packages is not writeable
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.6-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
Using cached httpcore-1.0.6-py3-none-any.whl (78 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
      Successfully uninstalled httpx-0.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the s

In [2]:
!pip install langsmith python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv('./.env')

True

In [4]:
import os

print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")
os.environ['LANGCHAIN_PROJECT'] = 'RAG_test'
print(f"[LANGCHAIN_PROJECT]\n{os.environ['LANGCHAIN_PROJECT']}")

[API KEY]
sk-l0rAOwEsqXqeXoZsj4jppfpH52TOFQezaNohE3ztURT3BlbkFJmqnZl5n_izzgDQoGAxMjXYm0nj4BrEzXWrArHJABUA
[LANGCHAIN_PROJECT]
RAG_test


API KEY 를 설정합니다.


In [5]:
pip install langchain-teddynote

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("test11")


LangSmith 추적을 시작합니다.
[프로젝트명]
test11


[LangSmith](https://smith.langchain.com)를 사용하여 체인이나 에이전트 내부에서 정확히 무슨 일이 일어나고 있는지 조사 가능


## 네이버 뉴스 기반 QA(Question-Answering) 챗봇

네이버 뉴스기사의 내용에 대해 질문할 수 있는 **뉴스기사 QA 앱** 을 구축할 것입니다.


In [7]:
!pip install langchain-community langchain_openai

Defaulting to user installation because normal site-packages is not writeable


In [8]:
### PDF 기반 QA(Question-Answering) 챗봇으로 변경하는 코드

from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 페이지 별 문서 로드
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")

ValueError: File path data/SPRI_AI_Brief_2023년12월호_F.pdf is not a valid file or url

In [ ]:
### csv 기반 QA(Question-Answering) 챗봇으로 변경하는 코드

from langchain_community.document_loaders.csv_loader import CSVLoader

# CSV 파일 로드
loader = CSVLoader(file_path="data/titanic.csv")
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")

In [ ]:
### 폴더 내의 모든 파일 로드하여 QA(Question-Answering) 챗봇으로 변경하는 코드

from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(".", glob="data/*.txt", show_progress=True)
docs = loader.load()

print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[0].page_content[:500]}")
print(f"\n[metadata]\n{docs[0].metadata}\n")


### 폴더 내의 모든 pdf 로드하여 QA(Question-Answering) 챗봇으로 변경하는 코드
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(".", glob="data/*.pdf")
docs = loader.load()

print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[2500:3000])

In [ ]:
### Python 기반 QA(Question-Answering) 챗봇으로 변경하는 코드

from langchain_community.document_loaders import PythonLoader

loader = DirectoryLoader(".", glob="**/*.py", loader_cls=PythonLoader)
docs = loader.load()

print(f"문서의 수: {len(docs)}\n")
print("[메타데이터]\n")
print(docs[0].metadata)
print("\n========= [앞부분] 미리보기 =========\n")
print(docs[0].page_content[:500])

In [ ]:
### txt 기반 QA(Question-Answering) 챗봇으로 변경하는 코드

from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/appendix-keywords.txt")
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[0].page_content[:500]}")
print(f"\n[metadata]\n{docs[0].metadata}\n")

In [9]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [10]:
! pip install pypdf faiss-cpu

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 96.1 MB/s eta 0:00:00:00:01


In [28]:
# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
loader = WebBaseLoader(
    web_paths=("https://jeong-daniel.github.io/posts/%EB%94%A5%EB%9F%AC%EB%8B%9D-%EB%AA%A8%EB%8D%B8%EC%9D%98-Backbone(%EB%B0%B1%EB%B3%B8)-%EC%A0%95%EB%A6%AC/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["post pl-1 pr-1 pl-md-2 pr-md-2"]},
        )
    ),
)

docs = loader.load()
print(f"문서의 수: {len(docs)}")
docs

문서의 수: 1


[Document(metadata={'source': 'https://jeong-daniel.github.io/posts/%EB%94%A5%EB%9F%AC%EB%8B%9D-%EB%AA%A8%EB%8D%B8%EC%9D%98-Backbone(%EB%B0%B1%EB%B3%B8)-%EC%A0%95%EB%A6%AC/'}, page_content='  딥러닝 모델의 Backbone(백본) 정리  Posted  Sep 20, 2022    Updated  Jan 8, 2023    By  Jeong Daniel    4 min read최근 과제를 하나 받아서 수행하고 있는데 백본 네트워크에 대해서 정리가 필요했다. 평소 컴퓨터비전 객체추적, 상황인식에 대해서 어느정도 알고 있었다고 생각했지만 또 역시 자세히 들여다보니 많은 부분으로 나누어져있다. 그래도 하나하나 짚어가다보면 끝이 보이겠지객체검출에 있어서는 One-Stage와 Two-Stage로 나누어지는데 객체 검출을 할때 두단계를 사용할 것인가 한단계로 통합할 것인가로 구분된다. 단계의 구분은 내가 원하는 물체가 있을 만한곳을 탐색하는 Location의 문제이고 그 물체가 어떤 것인지 구분하는 Classification으로 구분되는데 지금은 이것을 따지고자 쓴 게시글이 아니니 Backbone이 무엇인지부터 보자위 그림에서 처럼 이미지가 들어오면 Backbone - Neck - Head로 구분을 할 수 있다.Backbone(백본)은 입력 이미지를 feature map으로 변형시켜주는 부분이다. ImageNet 데이터셋으로 pre-trained 시킨 VGG16, ResNet-50 등이 대표적인 Backbone이다. 헤드는 Backbone에서 추출한 feature map의 location 작업을 수행하는 부분이다. 헤드에서 predict classes와 bounding boxes 작업이 수행된다.Neck(넥)은 Backbone과 Head를 연결하는 부분으로, feature map을 refinement(정제), reconfig

`RecursiveCharacterTextSplitter`는 문서를 지정된 크기의 청크로 나눔


In [29]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

3

`FAISS` 혹은 `Chroma`와 같은 vectorstore는 이러한 청크를 바탕으로 문서의 벡터 표현을 생성


In [30]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

`vectorstore.as_retriever()`를 통해 생성된 검색기는 프롬프트와 `ChatOpenAI` 모델을 사용하여 새로운 내용을 생성

`StrOutputParser`는 생성된 결과를 문자열로 파싱


In [35]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요. 줄바꿈을 해주세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

In [36]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

스트리밍 출력을 위하여 `stream_response` 를 사용

In [23]:
from langchain_teddynote.messages import stream_response



> [LangSmith Trace](https://smith.langchain.com/o/e738ca73-da9f-5fcd-86bb-d729db658172)


In [37]:
answer = rag_chain.stream("이미지 분석 기법")
# # 제너레이터에서 데이터를 하나씩 모아서 문자열로 합치기
# answer_content = ''.join([chunk for chunk in answer])

# # 결과 출력
# print(answer_content)
stream_response(answer)


이미지 분석 기법 중 하나인 딥러닝 모델의 Backbone(백본)은 입력 이미지를 feature map으로 변형시켜주는 역할을 합니다. 대표적인 Backbone으로는 ImageNet 데이터셋으로 사전 학습된 VGG16, ResNet-50 등이 있습니다. 

Backbone은 입력 이미지로부터 다양한 특징을 추출하는 feature extractor 역할을 하며, 이 추출된 feature는 이후 Neck(넥)과 Head(헤드)로 전달되어 객체 검출, 분류 등의 작업을 수행합니다. 

객체 검출에서는 Backbone이 추출한 feature map을 기반으로 Location과 Classification 작업이 이루어지며, 이러한 과정은 One-Stage와 Two-Stage로 나뉘어 진행됩니다. 

결론적으로, Backbone은 이미지 분석에서 중요한 역할을 하며, 사전 학습된 모델을 활용하여 성능을 향상시키는 데 기여합니다.

In [39]:
answer = rag_chain.stream("backbone으로 배경과 각종 물체를 분리하는 neck과 접힌 박막만 예측하는 head를 가진 tensorflow로 이루어진 코드를 부탁해. ")
stream_response(answer)

TensorFlow를 사용하여 Backbone으로 배경과 각종 물체를 분리하는 Neck과 접힌 박막만 예측하는 Head를 구현하는 코드는 다음과 같이 작성할 수 있습니다. 아래 코드는 간단한 구조를 보여주며, 실제로는 데이터셋과 모델의 세부 사항에 따라 조정이 필요할 수 있습니다.

```python
import tensorflow as tf
from tensorflow.keras import layers, models

# Backbone: 예를 들어 ResNet50을 사용
def create_backbone(input_shape):
    backbone = tf.keras.applications.ResNet50(input_shape=input_shape, include_top=False, weights='imagenet')
    return backbone

# Neck: Feature Pyramid Network (FPN) 예시
def create_neck(backbone_output):
    # FPN을 구현하는 간단한 예시
    x = layers.Conv2D(256, (1, 1), padding='same')(backbone_output)
    x = layers.UpSampling2D(size=(2, 2))(x)
    return x

# Head: 접힌 박막 예측을 위한 간단한 구조
def create_head(neck_output):
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(neck_output)
    x = layers.Conv2D(1, (1, 1), padding='same', activation='sigmoid')(x)  # 이진 분류를 위한 출력
    return x

# 전체 모델 생성
def create_model(input_shape):
    inputs = layers.Input(shape=input_shape)
    backbo

In [40]:
answer = rag_chain.stream("너가 준 코드에서 fit 하는 방법")
stream_response(answer)

주어진 코드에서 fit 하는 방법은 다음과 같습니다.

딥러닝 모델을 학습시키기 위해서는 일반적으로 `fit` 메서드를 사용합니다. 이 메서드는 모델에 데이터를 입력하고, 해당 데이터에 대해 학습을 수행합니다. 

1. **데이터 준비**: 먼저, 학습에 사용할 데이터셋을 준비해야 합니다. 이 데이터셋은 입력 데이터와 해당하는 레이블로 구성되어야 합니다.

2. **모델 정의**: 사용할 딥러닝 모델을 정의합니다. 이 모델은 Backbone을 포함하여 Neck과 Head로 구성될 수 있습니다.

3. **컴파일**: 모델을 컴파일하여 손실 함수와 최적화 알고리즘을 설정합니다.

4. **fit 호출**: 준비된 데이터셋을 사용하여 `fit` 메서드를 호출합니다. 이때, 에포크 수와 배치 크기 등의 하이퍼파라미터를 설정할 수 있습니다.

예를 들어, Keras를 사용하는 경우 다음과 같은 코드가 될 수 있습니다:

```python
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, train_labels, epochs=10, batch_size=32)
```

이렇게 하면 모델이 주어진 데이터에 대해 학습을 시작하게 됩니다.

In [41]:
answer = rag_chain.stream("backbone과 neck, head 각각 다른 라벨링 파일을 줄 필욘 없어?")
stream_response(answer)

Backbone, Neck, Head 각각에 대해 다른 라벨링 파일을 사용할 필요는 없습니다. 

이들은 딥러닝 모델의 구조에서 서로 연결된 부분으로, Backbone은 입력 이미지를 feature map으로 변형하고, Neck은 이 feature map을 정제 및 재구성하며, Head는 최종적으로 예측 작업을 수행합니다. 따라서 하나의 라벨링 파일로도 이 모든 부분을 처리할 수 있습니다. 

하지만 특정한 요구사항이나 데이터셋에 따라 다르게 설정할 수도 있으니, 필요에 따라 조정할 수 있습니다.